In [1]:
# IMPORTING LIBRARIES
import random
import numpy as np
import pandas as pd
from copy import deepcopy

# CONFIG SETUP
suits = {'C':"♣",
         'D':"♦",
         'H':"♥",
         'S':"♠"}

ranks  = {  "J":3,
            "9":2,
            "A":1,"10":1,
            "K":0,"Q":0
}
delim = "|"
num_players= 8
starting_player = 1

In [2]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.power = ranks[rank]

    def show(self):
        return self.rank + delim + self.suit


    

In [3]:
class Player:
    def __init__(self, play_n, init_hand):
        self.player_num = play_n
        self.hand = sorted(init_hand, key = lambda card: -1*card.power)

        self.team = play_n%2

    def pcard_sort(self, pcards):
        return sorted(pcards, key = lambda x: -1*x['card'].power)

    def strategy_1(self, round_cards, trump):
        if len(round_cards)==0:
            ### OPENING HAND
            sorted_hand = sorted(self.hand, key = lambda card: -1*card.power)

            if sorted_hand[0].rank == 'J':
                # If player has a J then play it
                play = sorted_hand[0]
            else:
                # Else play the lowest hand
                play = sorted_hand[-1]
            

        else:
            ### ONGOING ROUND
            reqd_suit = round_cards[0]['card'].suit
            sort_cards =sorted(round_cards, key = lambda play: -1*play['card'].power)


            top_card = sort_cards[0]
           
            vettus = self.pcard_sort([play for play in round_cards if play['card'].suit==trump])
            if len(vettus)>0: team_vettu = vettus[0]['player']%2==self.team
            else: team_vettu = False
            winning_team = (top_card['player']%2==0) or (team_vettu)
             

            # and (top_card[-1].power >= 2)
            # print(f"Team {self.team} winning: {winning_team}")

            if winning_team==self.team: func = max
            else: func = min

            # print(sort_cards)
            valid_cards = [card for card in self.hand if card.suit==reqd_suit]

            if len(valid_cards)!=0:
                # You HAVE a valid Suit Card
                if winning_team:
                    play = valid_cards[0]
                else:
                    play = valid_cards[-1]
                    
            else:
                # You DO NOT HAVE a valid Suit Card
                j = [card for card in self.hand if card.power==3]
                nonj = [card for card in self.hand if card.power<3]
                trumps = [card for card in self.hand if card.suit==trump]
                
                if winning_team:
                    if len(nonj)>0:
                        play = nonj[0]
                    else:
                        play = j[0]


                else:
                    if len(trumps)!=0:
                        play = trumps[-1]

                    elif len(nonj)!=0:
                        play = nonj[-1]

                    else:
                        play = j[-1]
                    
        self.hand.remove(play)
        return play
    

In [4]:
class Game:
    
    def __init__(self, num_players, show = False):
        ### STARTING THE GAME BY DIRTRIBUTING CARDS

        # Getting all cards set up
        cards = [[Card(suit, rank) for suit in suits.values()] for rank in ranks.keys()]
        # Converting List of Lists into a single List
        cards = [i for j in cards for i in j]

        if num_players >= 5: cards = cards*2

        
        # Shuffling the Cards
        random.shuffle(cards)
        
        # Splitting it to num_players in a dict
        self.players = {k+1: Player(k+1, list(hand)) for k, hand in enumerate(np.array_split(cards,num_players))}

        if show: self.show_hands()

        self.rounds = {}
        self.team_scores = {0:0, 1:0}


    def set_trump(self, trump):
        self.trump = trump
        print(f"\nTRUMP = {self.trump}")
    
    def show_hands(self):
        print("Current Hand of each Player")
        for player_num, player in self.players.items():
            print(str(player_num) + ") " + ", ".join([card.show() for card in player.hand]))
            
    def next_round_player(self, curr_round):
        start_card = curr_round[0]['card']
        start_suit = start_card.suit

        trumps = [play for play in curr_round if play['card'].suit==self.trump]
        vettu = len(trumps)>0
        if vettu:
            return sorted(trumps, key = lambda play: -1*play['card'].power)[0]['player']
        
        only_suit = sorted([play for play in curr_round if play['card'].suit==start_suit],
                           key = lambda play: -1*play['card'].power)
        return only_suit[0]['player'] 

    def round_statistics(self, curr_round):
        start_card = curr_round[0]['card']
        start_suit = start_card.suit

        trumps = [play for play in curr_round if play['card'].suit==self.trump]
        if len(trumps)>0: winning_team = sorted(trumps, key = lambda x: -1*x['card'].power)[0]['player']
        else: winning_team = sorted([pc for pc in curr_round if pc['card'].suit==start_suit], 
                                    key = lambda x: -1*x['card'].power)[0]['player']
            
        

        round_pts = sum([pc['card'].power for pc in curr_round])
        self.team_scores[winning_team%2]+= round_pts
        print("\n" + "\n".join([f"Team {k} Points: {v}" for k,v in self.team_scores.items()]))        


    def play_round(self, start_player=1):
        round_no = len(self.rounds.keys()) + 1
        print(f"\n\n\nRound {round_no}")

        round_stats = {}
        
        all_players = list(range(1,num_players+1))
        player_index = all_players.index(start_player)
        play_order = all_players[player_index:] + all_players[:player_index]

        self.round_cards = []
        for p,play in enumerate(play_order):
            play_card = self.players[play].strategy_1(self.round_cards, self.trump)
            print(f"{p+1}) Player {play}: {play_card.show()}")

            self.round_cards.append({'player':play, 'card':play_card})
        self.rounds[round_no] = self.round_cards

        self.round_statistics(self.round_cards)
        return self.next_round_player(self.round_cards)
    
    def play_game(self, round_starter=1):
        for k in range(1,int(56/num_players)):
            round_starter = self.play_round(round_starter)

    
            

In [11]:
new_game = Game(num_players, show = True)
game_stats = {0:{}, 1:{}}
for suit in suits.values():
    game_stats[0][suit] = {}; game_stats[1][suit] = {}
    for playn in range(1,num_players+1):
        game = deepcopy(new_game)
        game.set_trump(suit)
        game.play_game(playn)
        game_stats[0][suit][playn] = game.team_scores[0]
        game_stats[1][suit][playn] = game.team_scores[1]


display(pd.DataFrame(game_stats[0]).rename_axis('starter').T.style.background_gradient(cmap='Greens'))
new_game.show_hands()
display(pd.DataFrame(game_stats[1]).rename_axis('starter').T.style.background_gradient(cmap='Greens'))

Current Hand of each Player
1) 9|♣, 9|♦, 10|♦, 10|♣, 10|♦, Q|♦
2) J|♠, J|♥, A|♣, A|♦, 10|♥, A|♠
3) J|♦, J|♥, 9|♣, A|♥, K|♦, Q|♥
4) J|♣, J|♦, A|♥, K|♠, Q|♦, K|♥
5) 9|♥, 9|♠, 9|♠, 10|♥, K|♥, Q|♥
6) J|♣, 9|♦, A|♦, Q|♠, Q|♠, Q|♣
7) 9|♥, 10|♠, A|♠, A|♣, 10|♣, K|♣
8) J|♠, 10|♠, K|♣, K|♦, K|♠, Q|♣

TRUMP = ♣



Round 1
1) Player 1: Q|♦
2) Player 2: A|♦
3) Player 3: J|♦
4) Player 4: Q|♦
5) Player 5: Q|♥
6) Player 6: A|♦
7) Player 7: K|♣
8) Player 8: K|♦

Team 0 Points: 0
Team 1 Points: 5



Round 2
1) Player 7: 10|♣
2) Player 8: Q|♣
3) Player 1: 9|♣
4) Player 2: A|♣
5) Player 3: 9|♣
6) Player 4: J|♣
7) Player 5: 9|♥
8) Player 6: J|♣

Team 0 Points: 14
Team 1 Points: 5



Round 3
1) Player 4: J|♦
2) Player 5: 9|♠
3) Player 6: 9|♦
4) Player 7: 9|♥
5) Player 8: 10|♠
6) Player 1: 9|♦
7) Player 2: 10|♥
8) Player 3: K|♦

Team 0 Points: 27
Team 1 Points: 5



Round 4
1) Player 4: K|♥
2) Player 5: 10|♥
3) Player 6: Q|♣
4) Player 7: A|♣
5) Player 8: K|♣
6) Player 1: 10|♦
7) Player 2: J|♥
8) Player 3: J

starter,1,2,3,4,5,6,7,8
♣,27,24,26,44,36,32,43,25
♦,32,43,31,38,28,35,37,30
♥,16,32,15,18,20,18,18,21
♠,28,31,27,23,42,35,22,35


Current Hand of each Player
1) 9|♣, 9|♦, 10|♦, 10|♣, 10|♦, Q|♦
2) J|♠, J|♥, A|♣, A|♦, 10|♥, A|♠
3) J|♦, J|♥, 9|♣, A|♥, K|♦, Q|♥
4) J|♣, J|♦, A|♥, K|♠, Q|♦, K|♥
5) 9|♥, 9|♠, 9|♠, 10|♥, K|♥, Q|♥
6) J|♣, 9|♦, A|♦, Q|♠, Q|♠, Q|♣
7) 9|♥, 10|♠, A|♠, A|♣, 10|♣, K|♣
8) J|♠, 10|♠, K|♣, K|♦, K|♠, Q|♣


starter,1,2,3,4,5,6,7,8
♣,29,32,30,12,20,24,13,31
♦,24,13,25,18,28,21,19,26
♥,40,24,41,38,36,38,38,35
♠,28,25,29,33,14,21,34,21
